In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline 
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score

In [2]:
df=pd.read_csv(r"C:\Users\sarkar\Desktop\final\all about projects\churn prediction\insurance churn prediction\data\Insurance_Churn_ParticipantsData\Train.csv")

In [3]:
dffeature=df.copy()

In [4]:
dff=dffeature.drop(['feature_7','feature_10'],axis=1)

In [5]:
from sklearn.model_selection import train_test_split

In [8]:
X=dff.drop("labels",axis=1)
y=dff.labels

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
from sklearn.ensemble import RandomForestClassifier
model1=RandomForestClassifier()

In [11]:

model1.fit(X_train, y_train)

RandomForestClassifier()

In [12]:
predtrain=model1.predict(X_train)
predtest=model1.predict(X_test)

In [13]:
def performa(y,y_pred):
    confusion=confusion_matrix(y, y_pred)
    accuracy=accuracy_score(y, y_pred)
    precision=precision_score(y, y_pred)
    recall=recall_score(y, y_pred)
    ROC_AUC=roc_auc_score(y, y_pred)
    print("confusion_matrix\n",confusion)
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("ROC_AUC: ",ROC_AUC)

# Train

In [14]:
performa(y_train,predtrain)

confusion_matrix
 [[23956     0]
 [    0  3170]]
accuracy:  1.0
precision:  1.0
recall:  1.0
ROC_AUC:  1.0


# Test 

In [15]:
performa(y_test,predtest)

confusion_matrix
 [[5826  159]
 [ 482  315]]
accuracy:  0.9054851076378649
precision:  0.6645569620253164
recall:  0.39523212045169387
ROC_AUC:  0.6843328522057969


# Tuning 

In [16]:
#Using Grid Search
from sklearn.model_selection import GridSearchCV

In [17]:
#Using max_depth, criterion will suffice for DT Models, rest all will remain constant 
parameters = {'n_estimators' : (10,30,50,70,90,100)
              , 'criterion' : ('gini', 'entropy')
              , 'max_depth' : (3,5,7,9,10)
              , 'max_features' : ('auto', 'sqrt')
              , 'min_samples_split' : (2,4,6)
              #, 'min_weight_fraction_leaf' : (0.0,0.1,0.2,0.3)
             }

In [18]:
RF_grid  = GridSearchCV(RandomForestClassifier(n_jobs = -1, oob_score= False), param_grid = parameters, cv = 3, verbose = True)

In [19]:

RF_grid_model = RF_grid.fit(X_train, y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  7.9min finished


In [20]:
RF_grid_model.best_estimator_

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=6,
                       n_estimators=50, n_jobs=-1)

In [21]:
model2=RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=6,
                       n_estimators=50, n_jobs=-1)

In [22]:

model2.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=6,
                       n_estimators=50, n_jobs=-1)

In [23]:
predtrain=model2.predict(X_train)
predtest=model2.predict(X_test)

# Train

In [24]:
performa(y_train,predtrain)

confusion_matrix
 [[23794   162]
 [ 1810  1360]]
accuracy:  0.9273022192730221
precision:  0.8935611038107752
recall:  0.42902208201892744
ROC_AUC:  0.7111298421448786


In [25]:
performa(y_test,predtest)

confusion_matrix
 [[5893   92]
 [ 557  240]]
accuracy:  0.9043055145974639
precision:  0.7228915662650602
recall:  0.30112923462986196
ROC_AUC:  0.642878735944839


In [26]:
from sklearn.model_selection import cross_val_score
cross_val_score(RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=6,
                       n_estimators=50, n_jobs=-1), X_train, y_train,cv=5).mean()

0.9026394302224654

In [27]:
cross_val_score(RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_split=6,
                       n_estimators=50, n_jobs=-1), X_test, y_test,cv=5).mean()

0.9012090699812836

In [31]:
accuracy=[0.9043055145974639,0.9273022192730221]
precision=[0.7228915662650602,0.8935611038107752]
recall=[0.30112923462986196,0.42902208201892744]
ROC=[0.642878735944839,0.7111298421448786]
cross_val=[0.9012090699812836,0.9026394302224654]
df_final_performa=pd.DataFrame({"accuracy":accuracy,
                               "precision":precision,
                               "recall":recall,
                               "ROC":ROC,
                               "cross_val":cross_val},index={"test","train"})

In [32]:
df_final_performa

,accuracy,precision,recall,ROC,cross_val
test,0.904306,0.722892,0.301129,0.642879,0.901209
train,0.927302,0.893561,0.429022,0.711130,0.902639
